In [ ]:
import pandas as pd

# Step 1.1: Load dataset
file_path = '/content/Canada.csv'
df = pd.read_csv(file_path)

# Step 1.2: Show shape
print(f"✅ Dataset Shape: {df.shape}")




✅ Dataset Shape: (15369, 131)
🔍 Missing Value Percentage (Non-Zero Only):
School Program                                   100.000000
Number of Enrollments                            100.000000
Diversity Spotlight (US Headquarters Only)       100.000000
School Method                                    100.000000
Number of Private Contacts                       100.000000
Tags                                             100.000000
School Type                                      100.000000
Number of Private Notes                          100.000000
Number of Founders (Alumni)                       99.954454
Accelerator Application Deadline                  99.928427
Accelerator Duration (in weeks)                   99.915414
Accelerator Program Type                          99.908908
CB Rank (School)                                  99.895894
Valuation at IPO Currency (in USD)                99.785282
Valuation at IPO                                  99.785282
Valuation at IPO Currency 

/tmp/ipython-input-2974679807.py:5: DtypeWarning: Columns (101,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
# ✅ Step 2.1: Select 31 core columns based on the hackathon PDF/docx guidance
important_columns = [
    "Founded Date", "Number of Founders", "Company Type", "Number of Employees",
    "Industries", "Headquarters Location", "Headquarters Regions", "Number of Investors",
    "Actively Hiring", "Number of Funding Rounds", "Last Funding Amount",
    "Funding Status", "Last Funding Type", "Estimated Revenue Range", "IPqwery - Patents Granted",
    "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "SEMrush - Page Views / Visit",
    "SEMrush - Bounce Rate", "Number of Events", "BuiltWith - Active Tech Count",
    "G2 Stack - Total Products Active", "Number of Articles", "CB Rank (Company)",
    "Total Funding Amount", "Valuation at IPO", "Price", "Number of Exits",
    "Industry Groups", "Apptopia - Downloads Last 30 Days", "Apptopia - Number of Apps"
]

# Subset the dataframe
df_cleaned = df[important_columns].copy()

# Save to a new CSV working file
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Created working file: /content/Canada_cleaned.csv with 31 key columns")


✅ Created working file: /content/Canada_cleaned.csv with 31 key columns


In [ ]:
# Step 3.1: Fix CB Rank (Company) format
df_cleaned["CB Rank (Company)"] = df_cleaned["CB Rank (Company)"].astype(str).str.replace(",", "")
df_cleaned["CB Rank (Company)"] = pd.to_numeric(df_cleaned["CB Rank (Company)"], errors='coerce')

# Now fill missing values
print("Before:", df_cleaned['CB Rank (Company)'].isnull().sum())

# Grouped median by industry (if applicable)
if 'Industries' in df_cleaned.columns:
    df_cleaned['CB Rank (Company)'] = df_cleaned.groupby('Industries')["CB Rank (Company)"].transform(
        lambda x: x.fillna(x.median())
    )

# Fallback to global median
df_cleaned["CB Rank (Company)"] = df_cleaned["CB Rank (Company)"].fillna(df_cleaned["CB Rank (Company)"].median())

print("After:", df_cleaned['CB Rank (Company)'].isnull().sum())

# Save updated file
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ CB Rank (Company) cleaned and imputed.")


Before: 4
After: 0
✅ CB Rank (Company) cleaned and imputed.


In [ ]:
# Check if 'Founded Year' exists in the original dataset
if 'Founded Year' in df.columns:
    print("✅ 'Founded Year' exists. Using it to fill missing 'Founded Date'")

    # Convert 'Founded Date' to datetime
    df_cleaned['Founded Date'] = pd.to_datetime(df_cleaned['Founded Date'], errors='coerce')

    # Use 'Founded Year' to fill missing 'Founded Date'
    df_cleaned['Founded Date'] = df_cleaned['Founded Date'].fillna(
        pd.to_datetime(df['Founded Year'], errors='coerce')
    )

else:
    print("⚠️ 'Founded Year' not found. Will impute using group-wise median")

    # Convert 'Founded Date' to datetime
    df_cleaned['Founded Date'] = pd.to_datetime(df_cleaned['Founded Date'], errors='coerce')

    # Grouped median fill by Industry
    df_cleaned['Founded Date'] = df_cleaned.groupby('Industries')['Founded Date'].transform(
        lambda x: x.fillna(x.median())
    )

# Final fallback to global median
df_cleaned['Founded Date'] = df_cleaned['Founded Date'].fillna(df_cleaned['Founded Date'].median())

# Check result
print("✅ Founded Date Missing After Imputation:", df_cleaned['Founded Date'].isnull().sum())

# Save update
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Founded Date imputed and saved.")


⚠️ 'Founded Year' not found. Will impute using group-wise median


/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, ou

✅ Founded Date Missing After Imputation: 0
✅ Founded Date imputed and saved.


In [ ]:
# Step 3.3: Impute Company Type

print("Before:", df_cleaned["Company Type"].isnull().sum())

# Mode per industry group
df_cleaned["Company Type"] = df_cleaned.groupby("Industries")["Company Type"].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)

# Fallback global mode
global_mode = df_cleaned["Company Type"].mode().iloc[0]
df_cleaned["Company Type"] = df_cleaned["Company Type"].fillna(global_mode)

print("After:", df_cleaned["Company Type"].isnull().sum())

# Save changes
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Company Type imputed and saved.")


Before: 353
After: 0
✅ Company Type imputed and saved.


In [ ]:
# Step 3.4: Impute Industries
print("Before:", df_cleaned["Industries"].isnull().sum())

# Group-based mode (by Company Type)
df_cleaned["Industries"] = df_cleaned.groupby("Company Type")["Industries"].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)

# Fallback to global mode if still missing
global_mode = df_cleaned["Industries"].mode().iloc[0]
df_cleaned["Industries"] = df_cleaned["Industries"].fillna(global_mode)

print("After:", df_cleaned["Industries"].isnull().sum())

# Save update
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Industries imputed and saved.")


Before: 498
After: 0
✅ Industries imputed and saved.


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Step A: Clean "Number of Employees"
def parse_employees(val):
    try:
        val = str(val)
        if "-" in val:
            a, b = val.split("-")
            return (int(a.replace(",", "")) + int(b.replace(",", ""))) // 2
        elif val.isdigit():
            return int(val)
        else:
            return np.nan
    except:
        return np.nan

df_cleaned["Number of Employees"] = df_cleaned["Number of Employees"].apply(parse_employees)

# Step B: ML Imputation
features = [
    "Estimated Revenue Range", "SEMrush - Monthly Visits",
    "CB Rank (Company)", "Industries", "Funding Status"
]

train_df = df_cleaned[df_cleaned["Number of Employees"].notnull()].copy()
test_df = df_cleaned[df_cleaned["Number of Employees"].isnull()].copy()

# Encode categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_encoded = train_df[features].copy()
test_encoded = test_df[features].copy()

for col in train_encoded.columns:
    if train_encoded[col].dtype == "object":
        train_encoded[col] = train_encoded[col].astype(str)
        test_encoded[col] = test_encoded[col].astype(str)

encoder.fit(train_encoded)
X_train = encoder.transform(train_encoded)
X_test = encoder.transform(test_encoded)

y_train = train_df["Number of Employees"]

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing values
y_pred = model.predict(X_test)

# Fill in predictions
df_cleaned.loc[df_cleaned["Number of Employees"].isnull(), "Number of Employees"] = y_pred.astype(int)

# Save
print("✅ Missing After:", df_cleaned["Number of Employees"].isnull().sum())
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Number of Employees imputed with RandomForest and saved.")


✅ Missing After: 0
✅ Number of Employees imputed with RandomForest and saved.


In [ ]:
# Clean numeric columns with commas
numeric_fix_cols = ["Last Funding Amount", "SEMrush - Monthly Visits", "SEMrush - Visit Duration"]

for col in numeric_fix_cols:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(",", "")
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')


In [ ]:
print("🧪 Unique values in Estimated Revenue Range (non-null):")
print(df_cleaned["Estimated Revenue Range"].dropna().unique())


🧪 Unique values in Estimated Revenue Range (non-null):
['$1M to $10M' 'Less than $1M' '$10M to $50M' '$50M to $100M'
 '$100M to $500M' '$1B to $10B' '$500M to $1B' '$10B+']


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# === Step 0: Clean any comma-based numeric columns ===
numeric_fix_cols = ["Last Funding Amount", "SEMrush - Monthly Visits", "SEMrush - Visit Duration"]

for col in numeric_fix_cols:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(",", "")
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

# === Step 1: Define usable rows and clean unexpected labels ===
target_col = "Estimated Revenue Range"

# Define expected ordered labels
ordered_labels = [
    "Less than $1M", "$1M to $10M", "$10M to $50M",
    "$50M to $100M", "$100M to $500M", "$500M to $1B", "$1B+"
]

# Map all large values into final "catch-all" label
df_cleaned[target_col] = df_cleaned[target_col].replace({
    "$1B to $10B": "$1B+",
    "$10B+": "$1B+"
}).str.strip()

# Filter training and test sets
train_df = df_cleaned[df_cleaned[target_col].isin(ordered_labels)].copy()
test_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

# === Step 2: Label encode the target ===
label_mapping = {label: idx for idx, label in enumerate(ordered_labels)}
inv_label_map = {v: k for k, v in label_mapping.items()}

train_df[target_col] = train_df[target_col].map(label_mapping)

# === Step 3: Define features and encode them ===
features = [
    "Number of Employees", "CB Rank (Company)", "Funding Status",
    "Last Funding Amount", "SEMrush - Monthly Visits", "SEMrush - Visit Duration",
    "Industries"
]

cat_features = ["Funding Status", "Industries"]
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Encode categorical features
X_train = train_df[features].copy()
X_test = test_df[features].copy()

for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

encoder.fit(X_train[cat_features])
X_train[cat_features] = encoder.transform(X_train[cat_features])
X_test[cat_features] = encoder.transform(X_test[cat_features])

# === Step 4: Train the classifier ===
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, train_df[target_col])

# === Step 5: Predict and map back to original labels ===
y_pred = clf.predict(X_test)
y_pred_labels = [inv_label_map[int(p)] for p in y_pred]

# Fill the predicted labels into df_cleaned
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred_labels

# === Step 6: Save updated file ===
print("✅ Estimated Revenue Range missing after:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Estimated Revenue Range imputed and saved.")


✅ Estimated Revenue Range missing after: 0
✅ Estimated Revenue Range imputed and saved.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import pandas as pd

# === Step A: Clean commas and convert to float ===
df_cleaned["Last Funding Amount"] = df_cleaned["Last Funding Amount"].astype(str).str.replace(",", "")
df_cleaned["Last Funding Amount"] = pd.to_numeric(df_cleaned["Last Funding Amount"], errors='coerce')

# === Step B: Prepare training and test sets ===
target_col = "Last Funding Amount"

# Split based on availability
train_df = df_cleaned[df_cleaned[target_col].notnull()].copy()
test_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

# Features to use
features = [
    "Estimated Revenue Range", "Number of Employees", "CB Rank (Company)",
    "Industries", "Funding Status"
]

cat_features = ["Estimated Revenue Range", "Industries", "Funding Status"]

# Encode categorical columns
X_train = train_df[features].copy()
X_test = test_df[features].copy()

# Convert all categorical to string
for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# Ordinal encoding for categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(X_train[cat_features])
X_train[cat_features] = encoder.transform(X_train[cat_features])
X_test[cat_features] = encoder.transform(X_test[cat_features])

# Target variable
y_train = train_df[target_col]

# === Step C: Train the model ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === Step D: Predict missing values ===
y_pred = model.predict(X_test)

# Fill in predictions
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Save updated file
print("✅ Last Funding Amount missing after:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Last Funding Amount imputed and saved.")


✅ Last Funding Amount missing after: 0
✅ Last Funding Amount imputed and saved.


In [ ]:
print("🧪 Unique values in 'Funding Status' (non-null):")
print(df_cleaned["Funding Status"].dropna().unique())


🧪 Unique values in 'Funding Status' (non-null):
['Early Stage Venture' 'Seed' 'M&A' 'IPO' 'Late Stage Venture'
 'Private Equity']


In [ ]:




from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

# === Step A: Target column ===
target_col = "Funding Status"

# Train/test split
train_df = df_cleaned[df_cleaned[target_col].notnull()].copy()
test_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

# Features used (already cleaned/imputed)
features = [
    "Estimated Revenue Range", "Last Funding Amount", "CB Rank (Company)",
    "Number of Employees", "Industries"
]

cat_features = ["Estimated Revenue Range", "Industries"]

# Convert categoricals to string
X_train = train_df[features].copy()
X_test = test_df[features].copy()
for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# Ordinal encode categoricals
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
encoder.fit(X_train[cat_features])
X_train[cat_features] = encoder.transform(X_train[cat_features])
X_test[cat_features] = encoder.transform(X_test[cat_features])

# Prepare target
y_train = train_df[target_col].astype(str)

# === Step B: Train the model ===
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# === Step C: Predict and fill ===
y_pred = clf.predict(X_test)
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Save file
print("✅ Funding Status missing after:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Funding Status imputed and saved.")


✅ Funding Status missing after: 0
✅ Funding Status imputed and saved.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# === Step A: Clean column (remove commas if any) ===
df_cleaned["Total Funding Amount"] = df_cleaned["Total Funding Amount"].astype(str).str.replace(",", "")
df_cleaned["Total Funding Amount"] = pd.to_numeric(df_cleaned["Total Funding Amount"], errors='coerce')

# === Step B: Train/test split ===
target_col = "Total Funding Amount"
train_df = df_cleaned[df_cleaned[target_col].notnull()].copy()
test_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

# === Step C: Features for prediction ===
features = [
    "Last Funding Amount", "Estimated Revenue Range", "Number of Employees",
    "CB Rank (Company)", "Industries", "Funding Status"
]
cat_features = ["Estimated Revenue Range", "Industries", "Funding Status"]

# === Step D: Encode categorical features ===
X_train = train_df[features].copy()
X_test = test_df[features].copy()

for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
encoder.fit(X_train[cat_features])

X_train[cat_features] = encoder.transform(X_train[cat_features])
X_test[cat_features] = encoder.transform(X_test[cat_features])

y_train = train_df[target_col]

# === Step E: Train model ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === Step F: Predict and fill ===
y_pred = model.predict(X_test)
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# === Step G: Save file ===
print("✅ Total Funding Amount missing after:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print("✅ Total Funding Amount imputed and saved.")


✅ Total Funding Amount missing after: 0
✅ Total Funding Amount imputed and saved.


In [ ]:



from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# === Step A: Clean Valuation column ===
target_col = "Valuation at IPO"
df_cleaned[target_col] = df_cleaned[target_col].astype(str).str.replace(",", "")
df_cleaned[target_col] = pd.to_numeric(df_cleaned[target_col], errors='coerce')

# === Step B: Focus only on IPO companies ===
ipo_df = df_cleaned[df_cleaned["Funding Status"] == "IPO"].copy()

# Train/test split (within IPO rows only)
train_df = ipo_df[ipo_df[target_col].notnull()].copy()
test_df = ipo_df[ipo_df[target_col].isnull()].copy()

# If very few rows exist with actual values, just skip
if len(train_df) < 10:
    print("⚠️ Not enough IPO rows with real valuations. Skipping imputation.")
else:
    # === Step C: Define features ===
    features = [
        "Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range",
        "CB Rank (Company)", "Number of Employees", "Industries"
    ]
    cat_features = ["Estimated Revenue Range", "Industries"]

    X_train = train_df[features].copy()
    X_test = test_df[features].copy()

    for col in cat_features:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)

    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    encoder.fit(X_train[cat_features])
    X_train[cat_features] = encoder.transform(X_train[cat_features])
    X_test[cat_features] = encoder.transform(X_test[cat_features])

    y_train = train_df[target_col]

    # === Step D: Train model ===
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # === Step E: Predict and insert ===
    y_pred = model.predict(X_test)

    df_cleaned.loc[test_df.index, target_col] = y_pred

    # Save file
    print("✅ Valuation at IPO missing after:", df_cleaned[target_col].isnull().sum())
    df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
    print("✅ Valuation at IPO imputed and saved (only for IPO-tagged startups).")


✅ Valuation at IPO missing after: 13375
✅ Valuation at IPO imputed and saved (only for IPO-tagged startups).


In [ ]:
import pandas as pd

# Load cleaned file
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# Summary
print("✅ Shape:", df_cleaned.shape)
print("\n📉 Missing Values (%):")
print(df_cleaned.isnull().mean().sort_values(ascending=False) * 100)

# Show sample rows
df_cleaned.head()


✅ Shape: (15369, 31)

📉 Missing Values (%):
Actively Hiring                      99.232221
Number of Exits                      98.457935
Price                                97.800768
Apptopia - Downloads Last 30 Days    95.204633
Number of Events                     92.283167
Valuation at IPO                     87.025831
Apptopia - Number of Apps            84.761533
IPqwery - Patents Granted            76.010150
G2 Stack - Total Products Active     65.918407
Number of Investors                  64.890364
Number of Articles                   61.318238
SEMrush - Bounce Rate                59.431323
SEMrush - Page Views / Visit         59.431323
SEMrush - Visit Duration             59.431323
SEMrush - Monthly Visits             59.431323
Last Funding Type                    53.705511
Number of Funding Rounds             53.705511
Headquarters Regions                 53.445247
Number of Founders                   48.675906
BuiltWith - Active Tech Count        17.021277
Industry Groups 

,Founded Date,Number of Founders,Company Type,Number of Employees,Industries,Headquarters Location,Headquarters Regions,Number of Investors,Actively Hiring,Number of Funding Rounds,...,G2 Stack - Total Products Active,Number of Articles,CB Rank (Company),Total Funding Amount,Valuation at IPO,Price,Number of Exits,Industry Groups,Apptopia - Downloads Last 30 Days,Apptopia - Number of Apps
0,2014-01-01,NaN,For Profit,175.0,Software,"Vaughan, Ontario, Canada",Great Lakes,NaN,NaN,NaN,...,NaN,NaN,391288.5,4.044198e+06,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-05-01,NaN,For Profit,30.0,"Business Development, Digital Marketing, Profe...","Toronto, Ontario, Canada",Great Lakes,NaN,NaN,NaN,...,NaN,NaN,2477615.0,4.503167e+06,NaN,NaN,NaN,"Other, Professional Services, Sales and Marketing",NaN,NaN
2,2013-01-01,2.0,For Profit,75.0,"Content Creators, Marketing, Social Media, Soc...","Toronto, Ontario, Canada",Great Lakes,22.0,NaN,8.0,...,37.0,9,60890.0,2.141862e+07,NaN,NaN,NaN,"Internet Services, Media and Entertainment, Sa...",NaN,NaN
3,2016-01-01,NaN,For Profit,75.0,"Advertising, Digital Marketing, Marketing","Toronto, Ontario, Canada",Great Lakes,NaN,NaN,NaN,...,NaN,NaN,923626.0,2.904902e+07,NaN,NaN,NaN,"Advertising, Sales and Marketing",NaN,NaN
4,2021-01-01,NaN,For Profit,5.0,"Cloud Data Services, Internet","Causapscal, Quebec, Canada",NaN,NaN,NaN,NaN,...,NaN,NaN,1914268.0,4.187781e+06,NaN,NaN,NaN,"Information Technology, Internet Services",NaN,NaN


In [ ]:
col = "Industry Groups"

# A. Basic info
print(f"🔍 Column: {col}")
print(f"➡️ Missing: {df_cleaned[col].isnull().sum()} / {len(df_cleaned)}")
print(f"➡️ Missing %: {df_cleaned[col].isnull().mean()*100:.2f}%")

# B. Unique non-null values
print("\n🧪 Unique non-null values:")
print(df_cleaned[col].dropna().unique()[:30])  # show first 30 unique values

# C. Value counts (Top 10)
print("\n📊 Most common values:")
print(df_cleaned[col].value_counts().head(10))


🔍 Column: Industry Groups
➡️ Missing: 498 / 15369
➡️ Missing %: 3.24%

🧪 Unique non-null values:
['Other, Professional Services, Sales and Marketing'
 'Internet Services, Media and Entertainment, Sales and Marketing'
 'Advertising, Sales and Marketing'
 'Information Technology, Internet Services' 'Education'
 'Clothing and Apparel, Design'
 'Biotechnology, Content and Publishing, Media and Entertainment, Science and Engineering'
 'Professional Services, Sales and Marketing'
 'Information Technology, Internet Services, Professional Services, Software'
 'Advertising, Design, Internet Services, Sales and Marketing, Software'
 'Administrative Services, Professional Services'
 'Administrative Services, Apps, Information Technology, Mobile, Privacy and Security, Software'
 'Transportation' 'Professional Services' 'Education, Health Care, Sports'
 'Advertising, Content and Publishing, Media and Entertainment, Sales and Marketing'
 'Clothing and Apparel, Design, Information Technology, Interne

In [ ]:




# Only fill nulls using mode per 'Industries'
def fill_by_grouped_mode(df, target_col, group_col):
    df[target_col] = df.groupby(group_col)[target_col].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
    )
    return df

# Apply
df_cleaned = fill_by_grouped_mode(df_cleaned, "Industry Groups", "Industries")

# Confirm
print("✅ Missing after imputation:", df_cleaned["Industry Groups"].isnull().sum())

# Save
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# Step A: Setup
target_col = "BuiltWith - Active Tech Count"
features = [
    "Number of Employees", "Total Funding Amount",
    "Estimated Revenue Range", "Industry Groups"
]

# Step B: Prepare a shared encoder for both train and test
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Combine for encoding
non_null_rows = df_cleaned.dropna(subset=features)
encoder.fit(non_null_rows[["Estimated Revenue Range", "Industry Groups"]])

# Encode both train and predict parts
df_encoded = df_cleaned.copy()
df_encoded[["Estimated Revenue Range", "Industry Groups"]] = encoder.transform(
    df_encoded[["Estimated Revenue Range", "Industry Groups"]]
)

# Step C: Train the model
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step D: Predict and fill
pred_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = pred_df[features]
y_pred = model.predict(X_pred)

# Fill values into original dataframe
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Step E: Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
col = "Number of Founders"

# A. Basic info
print(f"🔍 Column: {col}")
print(f"➡️ Missing: {df_cleaned[col].isnull().sum()} / {len(df_cleaned)}")
print(f"➡️ Missing %: {df_cleaned[col].isnull().mean()*100:.2f}%")

# B. Unique values (non-null)
print("\n🧪 Unique non-null values:")
print(sorted(df_cleaned[col].dropna().unique()))

# C. Correlation analysis (optional preview)
print("\n📊 Correlation with:")
print(df_cleaned[[col, "Number of Employees", "Total Funding Amount"]].corr())


🔍 Column: Number of Founders
➡️ Missing: 7481 / 15369
➡️ Missing %: 48.68%

🧪 Unique non-null values:
[np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0), np.float64(14.0), np.float64(19.0), np.float64(20.0)]

📊 Correlation with:
                      Number of Founders  Number of Employees  \
Number of Founders              1.000000            -0.010644   
Number of Employees            -0.010644             1.000000   
Total Funding Amount            0.006965             0.451199   

                      Total Funding Amount  
Number of Founders                0.006965  
Number of Employees               0.451199  
Total Funding Amount              1.000000  


In [ ]:
col = "Number of Founders"
group_cols = ["Company Type", "Industries"]

# Step 1: Group-median logic
df_cleaned[col] = df_cleaned.groupby(group_cols)[col].transform(
    lambda x: x.fillna(x.median())
)

# Step 2: If any left, fill with global median
df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

# Step 3: Save
print("✅ Missing after imputation:", df_cleaned[col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:


import numpy as np

# Step A: Define a function to extract region
def extract_region(location):
    if pd.isnull(location): return np.nan
    parts = location.split(",")
    if len(parts) == 3:
        return parts[1].strip()  # Extract Province/State
    return np.nan

# Step B: Apply extraction
df_cleaned["Extracted Region"] = df_cleaned["Headquarters Location"].apply(extract_region)

# Step C: Fill missing values
df_cleaned["Headquarters Regions"] = df_cleaned["Headquarters Regions"].fillna(df_cleaned["Extracted Region"])

# Step D: Still missing? Fill with 'Unknown'
df_cleaned["Headquarters Regions"] = df_cleaned["Headquarters Regions"].fillna("Unknown")

# Step E: Drop helper column
df_cleaned.drop(columns=["Extracted Region"], inplace=True)

# Step F: Save
print("✅ Missing after imputation:", df_cleaned["Headquarters Regions"].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

target_col = "Number of Funding Rounds"
features = [
    "Total Funding Amount",
    "Last Funding Amount",
    "Estimated Revenue Range",
    "Funding Status",
    "Industries"
]

# Step A: Prepare Encoder
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded = df_cleaned.copy()
df_encoded[["Estimated Revenue Range", "Funding Status", "Industries"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Funding Status", "Industries"]]
)

# Step B: Train model
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step C: Predict
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)

# Step D: Impute
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Step E: Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
print(df_cleaned["Last Funding Type"].dropna().unique())


['Series B' 'Seed' 'Grant' 'Pre-Seed' 'Equity Crowdfunding'
 'Non-equity Assistance' 'Venture - Series Unknown' 'Angel' 'Series C'
 'Debt Financing' 'Private Equity' 'Series A' 'Undisclosed'
 'Post-IPO Equity' 'Post-IPO Debt' 'Corporate Round'
 'Product Crowdfunding' 'Series D' 'Convertible Note'
 'Initial Coin Offering' 'Series E' 'Secondary Market'
 'Post-IPO Secondary']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

target_col = "Last Funding Type"
features = [
    "Funding Status",
    "Estimated Revenue Range",
    "Total Funding Amount",
    "Number of Funding Rounds",
    "Industries",
    "Last Funding Amount"
]

# Step A: Ordinal Encode categorical features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded = df_cleaned.copy()
df_encoded[["Funding Status", "Estimated Revenue Range", "Industries"]] = encoder.fit_transform(
    df_encoded[["Funding Status", "Estimated Revenue Range", "Industries"]]
)

# Step B: Label encode the target
label_mapping = {label: idx for idx, label in enumerate(df_encoded[target_col].dropna().unique())}
inv_label_mapping = {v: k for k, v in label_mapping.items()}
df_encoded[target_col] = df_encoded[target_col].map(label_mapping)

# Step C: Prepare training and prediction sets
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict for missing
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = clf.predict(X_pred)

# Step D: Impute predicted values
predicted_labels = [inv_label_mapping[int(label)] for label in y_pred]
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = predicted_labels

# Step E: Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

target_col = "SEMrush - Monthly Visits"
features = [
    "Estimated Revenue Range",
    "Number of Employees",
    "Industries",
    "Last Funding Amount"
]

# Encode categorical features
df_encoded = df_cleaned.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded[["Estimated Revenue Range", "Industries"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Industries"]]
)

# Drop rows without any of the features
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict for missing
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)

# Impute values
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Save and confirm
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
# Target column
target_col = "SEMrush - Page Views / Visit"
features = [
    "Estimated Revenue Range",
    "Industries",
    "Number of Employees",
    "SEMrush - Monthly Visits"
]

# Encode categorical columns
df_encoded = df_cleaned.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded[["Estimated Revenue Range", "Industries"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Industries"]]
)

# Drop rows with missing feature or target
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)

# Fill missing
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Save and confirm
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
# Target
target_col = "SEMrush - Visit Duration"
features = [
    "SEMrush - Page Views / Visit",
    "SEMrush - Monthly Visits",
    "Estimated Revenue Range",
    "Industries"
]

# Ordinal encode
df_encoded = df_cleaned.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded[["Estimated Revenue Range", "Industries"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Industries"]]
)

# Training data
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

# Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)

# Impute
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
# Target column
target_col = "SEMrush - Bounce Rate"

# Step A: Preprocess percentage column
df_cleaned[target_col] = df_cleaned[target_col].apply(
    lambda x: float(str(x).replace('%', '')) if pd.notnull(x) else np.nan
)

# Features for prediction
features = [
    "SEMrush - Visit Duration",
    "SEMrush - Page Views / Visit",
    "Estimated Revenue Range",
    "Industries"
]

# Encode categoricals
df_encoded = df_cleaned.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded[["Estimated Revenue Range", "Industries"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Industries"]]
)

# Train data
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

# Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)

# Fill
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Optional: Reconvert to percentage string if needed
# df_cleaned[target_col] = df_cleaned[target_col].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else x)

# Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
print([col for col in df_cleaned.columns if "SEMrush" in col])


['SEMrush - Monthly Visits', 'SEMrush - Visit Duration', 'SEMrush - Page Views / Visit', 'SEMrush - Bounce Rate']


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import pandas as pd

# Target column
target_col = "Apptopia - Downloads Last 30 Days"

# Features
features = [
    "Apptopia - Number of Apps",
    "Estimated Revenue Range",
    "Industries",
    "Number of Employees",
    "Funding Status"
]

# Step 1: Encode categoricals
df_encoded = df_cleaned.copy()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_encoded[["Estimated Revenue Range", "Industries", "Funding Status"]] = encoder.fit_transform(
    df_encoded[["Estimated Revenue Range", "Industries", "Funding Status"]]
)

# Step 2: Clean target (remove commas, convert to float)
df_encoded[target_col] = df_encoded[target_col].astype(str).str.replace(",", "")
df_encoded[target_col] = pd.to_numeric(df_encoded[target_col], errors="coerce")

# Step 3: Filter training data
train_df = df_encoded.dropna(subset=features + [target_col])
X_train = train_df[features]
y_train = train_df[target_col]

# Step 4: Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict and fill missing values
predict_df = df_encoded[df_cleaned[target_col].isnull()]
X_pred = predict_df[features]
y_pred = model.predict(X_pred)
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# Step 6: Save
print("✅ Missing after imputation:", df_cleaned[target_col].isnull().sum())
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)


✅ Missing after imputation: 0


In [ ]:
print(df_cleaned["Apptopia - Number of Apps"].dropna().unique())


[  1.  16.   2.   7.   3.   4.  15.   5.   6.  32.   8.  19. 418.   9.
  10.  83.  11.  43.  20.  14.  12.  27.  26.  39.  76.  78.  30.  22.
 116.  56. 291. 268.  41.  42.  74.  17.  18.  21. 298.  52.  28.  13.
  45.  81.  40.  92.  29.  53.  33.  35.  23.  65.  46.  25.  66. 153.
  55.  38.  61.  50.  31. 235.   0.  36.  59. 191. 251.  90. 180. 198.
 129. 100.  24. 231. 241.]


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# 📂 Copy original cleaned dataset
df_encoded = df_cleaned.copy()

# 🎯 Target column to impute
target_col = "Apptopia - Number of Apps"

# 🔍 Features to use
features = [
    "Estimated Revenue Range",
    "Industries",
    "Funding Status",
    "Apptopia - Downloads Last 30 Days",
    "Number of Employees"
]

# 🧼 Convert target column to numeric (remove commas, convert to float)
df_encoded[target_col] = pd.to_numeric(df_encoded[target_col].astype(str).str.replace(",", ""), errors='coerce')

# 🧼 Also clean any numeric-looking features if needed
df_encoded["Apptopia - Downloads Last 30 Days"] = pd.to_numeric(
    df_encoded["Apptopia - Downloads Last 30 Days"].astype(str).str.replace(",", ""), errors='coerce'
)

# 🛑 Filter rows for training
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# 🧠 Categorical features
cat_features = ["Estimated Revenue Range", "Industries", "Funding Status"]

# 🔧 Fill missing and ensure strings
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna("Unknown")
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna("Unknown")

# 🧠 Encode categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# 🎯 Prepare training and prediction sets
X_train = train_df[features]
y_train = train_df[target_col].astype(float)
X_pred = predict_df[features]

# 🤖 Train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔮 Predict missing values
y_pred = model.predict(X_pred)

# 💾 Fill back into main DataFrame
df_encoded.loc[df_encoded[target_col].isnull(), target_col] = y_pred

# ✅ Update cleaned dataframe
df_cleaned[target_col] = df_encoded[target_col]

# 💾 Save to CSV
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)

# ✅ Final confirmation
print(f"✅ Missing after imputation: {df_cleaned[target_col].isnull().sum()}")


✅ Missing after imputation: 0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# 🛠️ Configuration
target_col = "SEMrush - Monthly Visits"   # change this to your target column
cat_features = ["Industry Groups", "Estimated Revenue Range", "Funding Status"]  # adjust as needed

# 🔍 Filter rows for training (non-null target)
train_df = df_cleaned[df_cleaned[target_col].notnull()].copy()
predict_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

# 🧼 Clean target column (remove %, commas, convert to float)
def clean_numeric_col(col):
    return (
        col.astype(str)
           .str.replace(",", "", regex=False)
           .str.replace("%", "", regex=False)
           .replace("nan", pd.NA)
           .astype(float)
    )

train_df[target_col] = clean_numeric_col(train_df[target_col])
df_cleaned[target_col] = clean_numeric_col(df_cleaned[target_col])

# 🔧 Fill missing and convert cat features to string for encoding
for col in cat_features:
    train_df[col] = train_df[col].astype(str).fillna("Unknown")
    predict_df[col] = predict_df[col].astype(str).fillna("Unknown")

# 🎯 Define features
features = cat_features

# 🔢 Encode categorical features safely
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])

if not predict_df.empty:
    predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# ✅ Model training
X_train = train_df[features]
y_train = train_df[target_col]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔮 Prediction and imputation
if not predict_df.empty:
    X_pred = predict_df[features]
    y_pred = model.predict(X_pred)
    df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred

# ✅ Final check
print(f"✅ Missing after imputation: {df_cleaned[target_col].isnull().sum()}")


✅ Missing after imputation: 0


In [ ]:
# 🧼 Check missing values count and percentage
missing_info = (
    df_cleaned.isnull().sum()
    .to_frame("Missing Count")
    .assign(
        Missing_Percentage=lambda x: (x["Missing Count"] / len(df_cleaned)) * 100
    )
    .sort_values(by="Missing Count", ascending=False)
)

# 🖨️ Show only columns with missing values
missing_info = missing_info[missing_info["Missing Count"] > 0]
print(missing_info)


                                  Missing Count  Missing_Percentage
Actively Hiring                           15251           99.232221
Number of Exits                           15132           98.457935
Price                                     15031           97.800768
Number of Events                          14183           92.283167
Valuation at IPO                          13375           87.025831
IPqwery - Patents Granted                 11682           76.010150
G2 Stack - Total Products Active          10131           65.918407
Number of Investors                        9973           64.890364
Number of Articles                         9424           61.318238


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# 🧹 Copy DataFrame to avoid modifying original
df_encoded = df_cleaned.copy()

# 🎯 Target column
target_col = "Number of Articles"

# 🧠 Features to predict the target
features = [
    "Number of Employees",
    "Total Funding Amount",
    "Estimated Revenue Range",
    "Industries",
    "Funding Status",
    "Number of Founders"
]

# 🧹 Clean target column: remove commas and convert to float
df_encoded[target_col] = df_encoded[target_col].astype(str).str.replace(",", "").replace("nan", pd.NA)
df_encoded[target_col] = pd.to_numeric(df_encoded[target_col], errors='coerce')

# 🎯 Drop rows with missing values in features or target (for training)
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# 🧠 Categorical features
cat_features = [col for col in features if df_encoded[col].dtype == 'object']

# Step 1: Fill NaNs with 'Unknown' and convert to string for encoding
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

# Step 2: Ordinal encoding
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# Step 3: Model training and prediction
X_train = train_df[features]
y_train = train_df[target_col]
X_pred = predict_df[features]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_pred)

# Step 4: Fill predicted values
df_encoded.loc[df_encoded[target_col].isnull(), target_col] = y_pred

# ✅ Update the main cleaned DataFrame
df_cleaned[target_col] = df_encoded[target_col]

# 💾 Save to CSV
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)

# ✅ Final check
print(f"✅ Missing after imputation: {df_cleaned[target_col].isnull().sum()}")


✅ Missing after imputation: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# ✅ Define target and features (REMOVE the missing column)
target_col = "Number of Investors"
features = [
    "Estimated Revenue Range",
    "Industries",
    "Funding Status",
    "Total Funding Amount",
    "Number of Funding Rounds",
    "Last Funding Amount"
]

# ✅ Work on a fresh copy
df_encoded = df_cleaned.copy()

# ✅ Handle numeric strings: remove commas and convert to float
for col in ["Total Funding Amount", "Last Funding Amount"]:
    df_encoded[col] = (
        df_encoded[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace("Unknown", np.nan)
        .astype(float)
    )

# ✅ Split training and prediction sets
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# ✅ Identify and encode categorical features
cat_features = ["Estimated Revenue Range", "Industries", "Funding Status"]
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna("Unknown")
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna("Unknown")

# ✅ Encode categorical columns
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# ✅ Prepare data for model
X_train = train_df[features]
y_train = train_df[target_col].astype(float)
X_pred = predict_df[features]

# ✅ Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ Predict
y_pred = model.predict(X_pred)

# ✅ Fill predictions back
df_encoded.loc[df_encoded[target_col].isnull(), target_col] = y_pred
df_cleaned[target_col] = df_encoded[target_col]

# ✅ Save file
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)

# ✅ Summary
missing_after = df_cleaned[target_col].isnull().sum()
print(f"✅ Missing after imputation: {missing_after}")


✅ Missing after imputation: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# 🧹 Step 1: Setup
df_encoded = df_cleaned.copy()
target_col = "G2 Stack - Total Products Active"

# 🧠 Step 2: Select features (add/remove based on your domain knowledge)
features = [
    "Estimated Revenue Range",
    "Industries",
    "Funding Status",
    "Number of Employees",
    "Total Funding Amount",
    "Apptopia - Number of Apps",
    "Apptopia - Downloads Last 30 Days"
]

# 🧪 Step 3: Drop rows with missing feature or target for training
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# 🧼 Step 4: Identify categorical features to encode
cat_features = train_df[features].select_dtypes(include='object').columns.tolist()

# 💡 Step 5: Preprocess categorical values
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')


# 🚀 Step 6: Encode categorical columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])


# 🧠 Step 7: Prepare train/predict sets
X_train = train_df[features]
y_train = train_df[target_col].astype(str).str.replace(",", "").astype(float)
X_pred = predict_df[features]

# 🏗️ Step 8: Train model and predict
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_pred)

# 💾 Step 9: Fill predicted values back into original dataframe
df_encoded.loc[df_encoded[target_col].isnull(), target_col] = y_pred
df_cleaned[target_col] = df_encoded[target_col]

# ✅ Step 10: Confirm
print(f"✅ Missing after imputation: {df_cleaned[target_col].isnull().sum()}")


✅ Missing after imputation: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

# Load the cleaned data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# --- Setup for Imputation ---
target_col = "Actively Hiring"
features = [
    "Number of Employees",
    "Estimated Revenue Range",
    "Funding Status",
    "Company Type"
]

# Create a copy for encoding
df_encoded = df_cleaned.copy()

# --- Preprocessing and Encoding ---
# Identify categorical features
cat_features = [col for col in features if df_encoded[col].dtype == 'object']

# Prepare training data: remove rows where target is missing
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# Handle 'nan' values in categorical features for encoding
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

# Use OrdinalEncoder to transform categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# --- Train and Predict ---
X_train = train_df[features]
y_train = train_df[target_col]
X_pred = predict_df[features]

# Train a RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the missing values
y_pred = model.predict(X_pred)

# --- Impute and Save ---
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)

print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")

✅ Missing after imputation for Actively Hiring: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# Load the cleaned data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# --- Setup for Imputation ---
target_col = "Number of Exits"

features = [
    "Total Funding Amount",
    "Number of Employees",
    "Funding Status",
    "Industries",
    "Company Type"
]

# Create a copy for encoding
df_encoded = df_cleaned.copy()

# --- Preprocessing and Encoding ---
# Ensure numeric features are floats
df_encoded['Total Funding Amount'] = pd.to_numeric(df_encoded['Total Funding Amount'], errors='coerce')
df_encoded['Number of Employees'] = pd.to_numeric(df_encoded['Number of Employees'], errors='coerce')

# Identify categorical features
cat_features = [col for col in features if df_encoded[col].dtype == 'object']

# Prepare training data: remove rows where target is missing
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# Handle 'nan' values in categorical features for encoding
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

# Use OrdinalEncoder to transform categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# --- Train and Predict ---
X_train = train_df[features]
y_train = train_df[target_col]
X_pred = predict_df[features]

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the missing values
y_pred = model.predict(X_pred)

# --- Impute and Save ---
# Fill with predicted values, rounded to the nearest integer
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = np.round(y_pred)
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)

print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")

✅ Missing after imputation for Number of Exits: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# Load the cleaned data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# --- Setup for Imputation ---
target_col = "Price"
features = [
    "Total Funding Amount",
    "Valuation at IPO",
    "Number of Employees",
    "Funding Status",
    "Industries"
]

# Create a copy for encoding
df_encoded = df_cleaned.copy()

# Filter for rows where a Price might exist (e.g., acquisitions, IPOs)
price_relevant_df = df_encoded[df_encoded['Funding Status'].isin(['M&A', 'IPO', 'Private Equity'])].copy()

# --- Preprocessing and Encoding ---
# Ensure numeric features are floats
for col in ["Total Funding Amount", "Valuation at IPO", "Number of Employees"]:
    price_relevant_df[col] = pd.to_numeric(price_relevant_df[col], errors='coerce')

# Identify categorical features
cat_features = [col for col in features if price_relevant_df[col].dtype == 'object']

# Split into training and prediction sets within the filtered data
train_df = price_relevant_df.dropna(subset=features + [target_col])
predict_df = price_relevant_df[price_relevant_df[target_col].isnull()]

if len(train_df) < 5:
    print(f"⚠️ Not enough training data for {target_col}. Imputation skipped.")
else:
    # Handle 'nan' values in categorical features for encoding
    for col in cat_features:
        train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
        predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

    # Use OrdinalEncoder to transform categorical features
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
    predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

    # --- Train and Predict ---
    X_train = train_df[features]
    y_train = train_df[target_col]
    X_pred = predict_df[features]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_pred)

    # --- Impute and Save ---
    df_cleaned.loc[predict_df.index, target_col] = y_pred

# Save the final cleaned dataframe
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)

print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")

⚠️ Not enough training data for Price. Imputation skipped.
✅ Missing after imputation for Price: 15031


In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# Load the cleaned data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# --- Setup for Imputation ---
target_col = "Number of Events"
features = [
    "Number of Employees",
    "Number of Articles",
    "Number of Funding Rounds",
    "Estimated Revenue Range",
    "Industries"
]

# Create a copy for encoding
df_encoded = df_cleaned.copy()

# --- Preprocessing and Encoding ---
# Ensure numeric features are floats
for col in ["Number of Employees", "Number of Articles", "Number of Funding Rounds"]:
    df_encoded[col] = pd.to_numeric(df_encoded[col], errors='coerce')

# Identify categorical features
cat_features = [col for col in features if df_encoded[col].dtype == 'object']

# Prepare training data: remove rows where target is missing
train_df = df_encoded.dropna(subset=features + [target_col])
predict_df = df_encoded[df_encoded[target_col].isnull()]

# Handle 'nan' values in categorical features for encoding
for col in cat_features:
    train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
    predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

# Use OrdinalEncoder to transform categorical features
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

# --- Train and Predict ---
X_train = train_df[features]
y_train = train_df[target_col]
X_pred = predict_df[features]

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the missing values
y_pred = model.predict(X_pred)

# --- Impute and Save ---
df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = np.round(y_pred)
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)

print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")

✅ Missing after imputation for Number of Events: 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# Load the cleaned data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# --- Step A: Clean Valuation column ---
target_col = "Valuation at IPO"
df_cleaned[target_col] = pd.to_numeric(
    df_cleaned[target_col].astype(str).str.replace(",", ""), errors='coerce'
)

# --- Step B: Focus only on IPO companies ---
ipo_df = df_cleaned[df_cleaned["Funding Status"] == "IPO"].copy()

# --- Step C: Prepare Data and Features ---
features = [
    "Total Funding Amount",
    "Last Funding Amount",
    "Estimated Revenue Range",
    "CB Rank (Company)",
    "Number of Employees",
    "Industries"
]
cat_features = ["Estimated Revenue Range", "Industries"]

train_df = ipo_df.dropna(subset=features + [target_col])
test_df = ipo_df[ipo_df[target_col].isnull()]

if len(train_df) < 5:
    print("⚠️ Not enough IPO rows with real valuations. Skipping imputation.")
elif test_df.empty:
    print(f"ℹ️ No missing values found for {target_col}. Nothing to predict.")
else:
    # Handle NaNs in categorical features
    for col in cat_features:
        train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
        test_df.loc[:, col] = test_df[col].astype(str).fillna('Unknown')

    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
    test_df.loc[:, cat_features] = encoder.transform(test_df[cat_features])

    X_train = train_df[features]
    y_train = train_df[target_col]
    X_test = test_df[features]

    # --- Step D: Train model ---
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # --- Step E: Predict and insert ---
    y_pred = model.predict(X_test)
    df_cleaned.loc[test_df.index, target_col] = y_pred
    print(f"✅ Imputed {len(test_df)} rows for {target_col}")

# Save file
df_cleaned.to_csv('/content/Canada_cleaned.csv', index=False)
print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")


ℹ️ No missing values found for Valuation at IPO. Nothing to predict.
✅ Missing after imputation for Valuation at IPO: 13375


In [ ]:
cols_to_check = [
    "Price",
    "Valuation at IPO",
    "IPqwery - Patents Granted",
    "G2 Stack - Total Products Active"
]

for col in cols_to_check:
    print(f"\n--- {col} ---")
    print("Unique count:", df_cleaned[col].nunique(dropna=False))
    print("Sample values:")
    print(df_cleaned[col].dropna().unique()[:20])  # first 20 unique non-null values




--- Price ---
Unique count: 247
Sample values:
[5.0000000e+07 1.7000000e+07 1.1000000e+08 1.0500000e+08 1.1000000e+09
 1.4884039e+07 4.7500000e+06 2.6600000e+08 9.0000000e+06 1.8000000e+06
 4.0000000e+08 1.2000000e+07 2.7000000e+06 2.2500000e+08 5.0000000e+06
 4.7000000e+07 2.1200000e+06 3.6000000e+07 1.1500000e+08 7.5000000e+07]

--- Valuation at IPO ---
Unique count: 1348
Sample values:
[7.74767914e+07 7.74855614e+07 3.36503026e+08 3.00833658e+08
 9.94283827e+07 1.83602552e+08 1.45835925e+08 1.20607788e+08
 7.21985942e+07 9.53598513e+07 6.36624351e+08 9.75453778e+07
 1.21036488e+08 5.52377639e+08 1.45626652e+08 1.28890038e+08
 1.07132846e+08 7.04065800e+07 7.75726914e+07 9.36664814e+07]

--- IPqwery - Patents Granted ---
Unique count: 107
Sample values:
['0.0' '1.0' '0' '67' '2.0' '9.0' '22.0' '12.0' '4.0' '13.0' '7.0' '6'
 '8.0' '3' '19.0' '6.0' '2' '16.0' '3.0' '37.0']

--- G2 Stack - Total Products Active ---
Unique count: 83
Sample values:
[37.  8. 18.  6. 16. 13. 21.  7. 17. 31

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

# Load data
df_cleaned = pd.read_csv('/content/Canada_cleaned.csv')

# List of numeric target columns to impute
target_cols = [
    "Price",
    "Valuation at IPO",
    "IPqwery - Patents Granted",
    "G2 Stack - Total Products Active"
]

# Features we can use for prediction (choose generic ones available for all rows)
base_features = [
    "Total Funding Amount",
    "Last Funding Amount",
    "Estimated Revenue Range",
    "CB Rank (Company)",
    "Number of Employees",
    "Industries"
]

cat_features = ["Estimated Revenue Range", "Industries"]

for target_col in target_cols:
    print(f"\n🚀 Processing {target_col} ...")

    # Step 1: Force numeric
    df_cleaned[target_col] = pd.to_numeric(
        df_cleaned[target_col].astype(str).str.replace(",", "").str.strip(),
        errors='coerce'
    )

    # Step 2: Train/predict split
    train_df = df_cleaned.dropna(subset=base_features + [target_col]).copy()
    test_df = df_cleaned[df_cleaned[target_col].isnull()].copy()

    if len(train_df) < 5:
        print(f"⚠️ Skipping {target_col} — not enough rows to train ({len(train_df)} available).")
        continue

    # Step 3: Handle categoricals
    for col in cat_features:
        train_df.loc[:, col] = train_df[col].astype(str).fillna("Unknown")
        test_df.loc[:, col] = test_df[col].astype(str).fillna("Unknown")

    encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
    test_df.loc[:, cat_features] = encoder.transform(test_df[cat_features])

    # Step 4: Train model
    X_train = train_df[base_features]
    y_train = train_df[target_col]
    X_test = test_df[base_features]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Step 5: Predict and insert
    preds = model.predict(X_test)
    df_cleaned.loc[test_df.index, target_col] = preds

    print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")

# Save results
df_cleaned.to_csv("/content/Canada_cleaned.csv", index=False)
print("\n💾 All imputations complete and saved to /content/Canada_cleaned.csv")



🚀 Processing Price ...
✅ Missing after imputation for Price: 0

🚀 Processing Valuation at IPO ...
✅ Missing after imputation for Valuation at IPO: 0

🚀 Processing IPqwery - Patents Granted ...
✅ Missing after imputation for IPqwery - Patents Granted: 0

🚀 Processing G2 Stack - Total Products Active ...
✅ Missing after imputation for G2 Stack - Total Products Active: 0

💾 All imputations complete and saved to /content/Canada_cleaned.csv


In [2]:
# 🧼 Check missing values count and percentage
missing_info = (
    df_cleaned.isnull().sum()
    .to_frame("Missing Count")
    .assign(
        Missing_Percentage=lambda x: (x["Missing Count"] / len(df_cleaned)) * 100
    )
    .sort_values(by="Missing Count", ascending=False)
)

# 🖨️ Show only columns with missing values
missing_info = missing_info[missing_info["Missing Count"] > 0]
print(missing_info)



NameError: name 'df_cleaned' is not defined

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import warnings

warnings.filterwarnings('ignore')

def process_country_data(country_name, file_path):
    """
    Cleans and imputes missing values for a given country's dataset.

    Args:
        country_name (str): The name of the country.
        file_path (str): The path to the country's CSV file.
    """
    print(f"--- 🌎 Processing {country_name} data... ---")

    # Step 1: Load dataset and select core columns
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Error: {file_path} not found. Skipping {country_name}.")
        return

    important_columns = [
        "Founded Date", "Number of Founders", "Company Type", "Number of Employees",
        "Industries", "Headquarters Location", "Headquarters Regions", "Number of Investors",
        "Actively Hiring", "Number of Funding Rounds", "Last Funding Amount",
        "Funding Status", "Last Funding Type", "Estimated Revenue Range", "IPqwery - Patents Granted",
        "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "SEMrush - Page Views / Visit",
        "SEMrush - Bounce Rate", "Number of Events", "BuiltWith - Active Tech Count",
        "G2 Stack - Total Products Active", "Number of Articles", "CB Rank (Company)",
        "Total Funding Amount", "Valuation at IPO", "Price", "Number of Exits",
        "Industry Groups", "Apptopia - Downloads Last 30 Days", "Apptopia - Number of Apps"
    ]

    # Ensure all columns exist before trying to subset
    available_cols = [col for col in important_columns if col in df.columns]
    df_cleaned = df[available_cols].copy()

    # Add back any missing columns with NaN values to maintain a consistent structure
    for col in important_columns:
        if col not in df_cleaned.columns:
            df_cleaned[col] = np.nan

    print(f"✅ Loaded data with shape: {df_cleaned.shape}")

    # Helper functions for imputation
    def clean_numeric_col(series):
        return pd.to_numeric(
            series.astype(str).str.replace('[,%$]', '', regex=True).str.strip(),
            errors='coerce'
        )

    def fill_by_grouped_mode(df, target_col, group_col):
        if target_col in df.columns and group_col in df.columns:
            df[target_col] = df.groupby(group_col)[target_col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )
        return df

    def fill_by_grouped_median(df, target_col, group_col):
        if target_col in df.columns and group_col in df.columns:
            df[target_col] = df.groupby(group_col)[target_col].transform(
                lambda x: x.fillna(x.median())
            )
        return df

    # Step 2: Impute columns with low missing values or simple logic

    # CB Rank (Company)
    if "CB Rank (Company)" in df_cleaned.columns:
        df_cleaned["CB Rank (Company)"] = clean_numeric_col(df_cleaned["CB Rank (Company)"])
        df_cleaned = fill_by_grouped_median(df_cleaned, "CB Rank (Company)", "Industries")
        df_cleaned["CB Rank (Company)"].fillna(df_cleaned["CB Rank (Company)"].median(), inplace=True)
        print("✅ Imputed CB Rank (Company).")

    # Founded Date
    if "Founded Date" in df_cleaned.columns:
        df_cleaned['Founded Date'] = pd.to_datetime(df_cleaned['Founded Date'], errors='coerce')
        df_cleaned = fill_by_grouped_median(df_cleaned, "Founded Date", "Industries")
        df_cleaned['Founded Date'].fillna(df_cleaned['Founded Date'].median(), inplace=True)
        print("✅ Imputed Founded Date.")

    # Company Type
    if "Company Type" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Company Type", "Industries")
        df_cleaned["Company Type"].fillna(df_cleaned["Company Type"].mode().iloc[0], inplace=True)
        print("✅ Imputed Company Type.")

    # Industries
    if "Industries" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Industries", "Company Type")
        df_cleaned["Industries"].fillna(df_cleaned["Industries"].mode().iloc[0], inplace=True)
        print("✅ Imputed Industries.")

    # Number of Employees
    if "Number of Employees" in df_cleaned.columns:
        def parse_employees(val):
            try:
                val = str(val).replace(',', '')
                if "nan" in val: return np.nan
                if "less than" in val.lower(): return 1
                if "-" in val:
                    a, b = val.split("-")
                    return (int(a) + int(b)) // 2
                return int(val)
            except:
                return np.nan

        df_cleaned["Number of Employees"] = df_cleaned["Number of Employees"].apply(parse_employees)
        df_cleaned["Number of Employees"].fillna(df_cleaned["Number of Employees"].median(), inplace=True)
        print("✅ Imputed Number of Employees.")

    # Industry Groups
    if "Industry Groups" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Industry Groups", "Industries")
        df_cleaned["Industry Groups"].fillna(df_cleaned["Industry Groups"].mode().iloc[0], inplace=True)
        print("✅ Imputed Industry Groups.")

    # Headquarters Regions
    if "Headquarters Regions" in df_cleaned.columns:
        def extract_region(location):
            if pd.isnull(location): return np.nan
            parts = location.split(",")
            if len(parts) == 3:
                return parts[1].strip()
            return np.nan

        df_cleaned['Extracted Region'] = df_cleaned['Headquarters Location'].apply(extract_region)
        df_cleaned['Headquarters Regions'].fillna(df_cleaned['Extracted Region'], inplace=True)
        df_cleaned['Headquarters Regions'].fillna('Unknown', inplace=True)
        df_cleaned.drop(columns=['Extracted Region'], inplace=True, errors='ignore')
        print("✅ Imputed Headquarters Regions.")

    # Step 3: Impute using Machine Learning models

    # Imputation function for regression
    def ml_impute_regressor(df, target_col, features):
        df_encoded = df.copy()
        df_encoded[target_col] = clean_numeric_col(df_encoded[target_col])

        cat_features = [col for col in features if df_encoded[col].dtype == 'object']

        train_df = df_encoded.dropna(subset=features + [target_col])
        predict_df = df_encoded[df_encoded[target_col].isnull()]

        if len(train_df) < 5 or predict_df.empty:
            print(f"ℹ️ Skipping ML imputation for {target_col} due to insufficient data.")
            if predict_df.empty:
                print("✅ All values are already present for this column.")
            return df

        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

        # Preprocess and encode
        for col in cat_features:
            train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
            predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

        train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
        predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(train_df[features], train_df[target_col])
        y_pred = model.predict(predict_df[features])

        df.loc[predict_df.index, target_col] = y_pred
        return df

    # Imputation function for classification
    def ml_impute_classifier(df, target_col, features):
        df_encoded = df.copy()

        cat_features = [col for col in features if df_encoded[col].dtype == 'object']

        train_df = df_encoded.dropna(subset=features + [target_col])
        predict_df = df_encoded[df_encoded[target_col].isnull()]

        if len(train_df) < 5 or predict_df.empty:
            print(f"ℹ️ Skipping ML imputation for {target_col} due to insufficient data.")
            if predict_df.empty:
                print("✅ All values are already present for this column.")
            return df

        # Preprocess and encode
        for col in cat_features:
            train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
            predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
        predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(train_df[features], train_df[target_col])
        y_pred = model.predict(predict_df[features])

        df.loc[predict_df.index, target_col] = y_pred
        return df

    # List of columns to impute using ML
    impute_list_reg = {
        "Estimated Revenue Range": ["Number of Employees", "CB Rank (Company)", "Funding Status", "Last Funding Amount", "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "Industries"],
        "Last Funding Amount": ["Estimated Revenue Range", "Number of Employees", "CB Rank (Company)", "Industries", "Funding Status"],
        "Total Funding Amount": ["Last Funding Amount", "Estimated Revenue Range", "Number of Employees", "CB Rank (Company)", "Industries", "Funding Status"],
        "Valuation at IPO": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Number of Articles": ["Number of Employees", "Total Funding Amount", "Estimated Revenue Range", "Industries", "Funding Status", "Number of Founders"],
        "Number of Investors": ["Estimated Revenue Range", "Industries", "Funding Status", "Total Funding Amount", "Number of Funding Rounds", "Last Funding Amount"],
        "BuiltWith - Active Tech Count": ["Number of Employees", "Total Funding Amount", "Estimated Revenue Range", "Industry Groups"],
        "G2 Stack - Total Products Active": ["Estimated Revenue Range", "Industries", "Funding Status", "Number of Employees", "Total Funding Amount"],
        "IPqwery - Patents Granted": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Number of Events": ["Number of Employees", "Number of Articles", "Number of Funding Rounds", "Estimated Revenue Range", "Industries"],
        "Number of Exits": ["Total Funding Amount", "Number of Employees", "Funding Status", "Industries", "Company Type"],
        "Price": ["Total Funding Amount", "Valuation at IPO", "Number of Employees", "Funding Status", "Industries"],
        "Number of Funding Rounds": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "Funding Status", "Industries"],
        "Number of Founders": ["Company Type", "Industries", "Number of Employees", "Total Funding Amount", "Number of Articles"]
    }

    impute_list_class = {
        "Funding Status": ["Estimated Revenue Range", "Last Funding Amount", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Last Funding Type": ["Funding Status", "Estimated Revenue Range", "Total Funding Amount", "Number of Funding Rounds", "Industries", "Last Funding Amount"],
        "Actively Hiring": ["Number of Employees", "Estimated Revenue Range", "Funding Status", "Company Type"]
    }

    # Special case for SEMrush and Apptopia columns
    semrush_features = ["Estimated Revenue Range", "Industries", "Number of Employees", "Last Funding Amount"]
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Monthly Visits", semrush_features)
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Page Views / Visit", semrush_features + ["SEMrush - Monthly Visits"])
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Visit Duration", semrush_features + ["SEMrush - Page Views / Visit"])
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Bounce Rate", semrush_features + ["SEMrush - Visit Duration"])

    apptopia_features = ["Estimated Revenue Range", "Industries", "Funding Status", "Number of Employees", "Apptopia - Number of Apps"]
    df_cleaned = ml_impute_regressor(df_cleaned, "Apptopia - Downloads Last 30 Days", apptopia_features)
    df_cleaned = ml_impute_regressor(df_cleaned, "Apptopia - Number of Apps", apptopia_features + ["Apptopia - Downloads Last 30 Days"])

    # Run ML imputation for all other columns
    for col, features in impute_list_reg.items():
        if col in df_cleaned.columns and all(f in df_cleaned.columns for f in features):
            df_cleaned = ml_impute_regressor(df_cleaned, col, features)
            print(f"✅ Imputed {col} using a regressor.")

    for col, features in impute_list_class.items():
        if col in df_cleaned.columns and all(f in df_cleaned.columns for f in features):
            df_cleaned = ml_impute_classifier(df_cleaned, col, features)
            print(f"✅ Imputed {col} using a classifier.")

    # Final check and save
    final_output_path = f"/content/{country_name}_final_filled.csv"
    df_cleaned.to_csv(final_output_path, index=False)
    print(f"✅ Final cleaned data for {country_name} saved to {final_output_path}")
    print(f"--- 🏁 {country_name} processing complete. ---\n")

# List of countries to process
countries_to_process = [
    ('Algeria', 'Algeria.csv'),
    ('Angola', 'Angola.csv'),
    ('Armenia', 'Armenia.csv'),
    ('Australia', 'Australia.csv'),
    ('Austria', 'Austria.csv'),
    ('Belarus', 'Belarus.csv'),
    ('Belize', 'Belize.csv'),
    ('British_Indian_Ocean_Territory', 'British_Indian_Ocean_Territory.csv'),
    ('Cameroon', 'Cameroon.csv'),
    ('Canada', 'Canada.csv'),
    ('Colombia', 'Colombia.csv'),
    ('Congo', 'Congo.csv'),
    ('Cyprus', 'Cyprus.csv'),
    ('Ecuador', 'Ecuador.csv'),
    ('Egypt', 'Egypt.csv'),
    ('Equatorial_Guinea', 'Equatorial_Guinea.csv'),
    ('Ethiopia', 'Ethiopia.csv'),
    ('Faroe_Islands', 'Faroe_Islands.csv'),
    ('Fiji', 'Fiji.csv'),
    ('Finland', 'Finland.csv'),
    ('Gabon', 'Gabon.csv'),
    ('Ghana', 'Ghana.csv'),
    ('Greece', 'Greece.csv'),
    ('Guinea', 'Guinea.csv'),
    ('Guyana', 'Guyana.csv'),
    ('Honduras', 'Honduras.csv'),
    ('Iceland', 'Iceland.csv'),
    ('Indonesia', 'Indonesia.csv'),
    ('Iraq', 'Iraq.csv'),
    ('Isle_of_Man', 'Isle_of_Man.csv'),
    ('Jersey', 'Jersey.csv'),
    ('Malawi', 'Malawi.csv'),
    ('Marshall_Islands', 'Marshall_Islands.csv'),
    ('Mauritania', 'Mauritania.csv'),
    ('Mauritius', 'Mauritius.csv')
]

# Run the process for each country
# for country, file in countries_to_process:
#     process_country_data(country, f"/content/{file}")
process_country_data('Cyprus', "Cyprus.csv")

--- 🌎 Processing Cyprus data... ---
✅ Loaded data with shape: (1220, 31)
✅ Imputed CB Rank (Company).
✅ Imputed Founded Date.
✅ Imputed Company Type.
✅ Imputed Industries.
✅ Imputed Number of Employees.
✅ Imputed Industry Groups.
✅ Imputed Headquarters Regions.
ℹ️ Skipping ML imputation for Estimated Revenue Range due to insufficient data.
✅ Imputed Estimated Revenue Range using a regressor.
✅ Imputed Last Funding Amount using a regressor.
✅ Imputed Total Funding Amount using a regressor.
ℹ️ Skipping ML imputation for Valuation at IPO due to insufficient data.
✅ Imputed Valuation at IPO using a regressor.
✅ Imputed Number of Articles using a regressor.
✅ Imputed Number of Investors using a regressor.
✅ Imputed BuiltWith - Active Tech Count using a regressor.
✅ Imputed G2 Stack - Total Products Active using a regressor.
✅ Imputed IPqwery - Patents Granted using a regressor.
✅ Imputed Number of Events using a regressor.
ℹ️ Skipping ML imputation for Number of Exits due to insufficient da

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import warnings

warnings.filterwarnings('ignore')

def process_country_data(country_name, file_path):
    """
    Cleans and imputes missing values for a given country's dataset.

    Args:
        country_name (str): The name of the country.
        file_path (str): The path to the country's CSV file.
    """
    print(f"--- 🌎 Processing {country_name} data... ---")

    # Step 1: Load dataset and select core columns
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Error: {file_path} not found. Skipping {country_name}.")
        return

    important_columns = [
        "Founded Date", "Number of Founders", "Company Type", "Number of Employees",
        "Industries", "Headquarters Location", "Headquarters Regions", "Number of Investors",
        "Actively Hiring", "Number of Funding Rounds", "Last Funding Amount",
        "Funding Status", "Last Funding Type", "Estimated Revenue Range", "IPqwery - Patents Granted",
        "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "SEMrush - Page Views / Visit",
        "SEMrush - Bounce Rate", "Number of Events", "BuiltWith - Active Tech Count",
        "G2 Stack - Total Products Active", "Number of Articles", "CB Rank (Company)",
        "Total Funding Amount", "Valuation at IPO", "Price", "Number of Exits",
        "Industry Groups", "Apptopia - Downloads Last 30 Days", "Apptopia - Number of Apps"
    ]

    # Ensure all columns exist before trying to subset
    available_cols = [col for col in important_columns if col in df.columns]
    df_cleaned = df[available_cols].copy()

    # Add back any missing columns with NaN values to maintain a consistent structure
    for col in important_columns:
        if col not in df_cleaned.columns:
            df_cleaned[col] = np.nan

    print(f"✅ Loaded data with shape: {df_cleaned.shape}")

    # Helper functions for imputation
    def clean_numeric_col(series):
        return pd.to_numeric(
            series.astype(str).str.replace('[,%$]', '', regex=True).str.strip(),
            errors='coerce'
        )

    def fill_by_grouped_mode(df, target_col, group_col):
        if target_col in df.columns and group_col in df.columns:
            df[target_col] = df.groupby(group_col)[target_col].transform(
                lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
            )
        return df

    def fill_by_grouped_median(df, target_col, group_col):
        if target_col in df.columns and group_col in df.columns:
            df[target_col] = df.groupby(group_col)[target_col].transform(
                lambda x: x.fillna(x.median())
            )
        return df

    # Step 2: Impute columns with low missing values or simple logic

    # CB Rank (Company)
    if "CB Rank (Company)" in df_cleaned.columns:
        df_cleaned["CB Rank (Company)"] = clean_numeric_col(df_cleaned["CB Rank (Company)"])
        df_cleaned = fill_by_grouped_median(df_cleaned, "CB Rank (Company)", "Industries")
        df_cleaned["CB Rank (Company)"].fillna(df_cleaned["CB Rank (Company)"].median(), inplace=True)
        print("✅ Imputed CB Rank (Company).")

    # Founded Date
    if "Founded Date" in df_cleaned.columns:
        df_cleaned['Founded Date'] = pd.to_datetime(df_cleaned['Founded Date'], errors='coerce')
        df_cleaned = fill_by_grouped_median(df_cleaned, "Founded Date", "Industries")
        df_cleaned['Founded Date'].fillna(df_cleaned['Founded Date'].median(), inplace=True)
        print("✅ Imputed Founded Date.")

    # Company Type
    if "Company Type" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Company Type", "Industries")
        df_cleaned["Company Type"].fillna(df_cleaned["Company Type"].mode().iloc[0], inplace=True)
        print("✅ Imputed Company Type.")

    # Industries
    if "Industries" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Industries", "Company Type")
        df_cleaned["Industries"].fillna(df_cleaned["Industries"].mode().iloc[0], inplace=True)
        print("✅ Imputed Industries.")

    # Number of Employees
    if "Number of Employees" in df_cleaned.columns:
        def parse_employees(val):
            try:
                val = str(val).replace(',', '')
                if "nan" in val: return np.nan
                if "less than" in val.lower(): return 1
                if "-" in val:
                    a, b = val.split("-")
                    return (int(a) + int(b)) // 2
                return int(val)
            except:
                return np.nan

        df_cleaned["Number of Employees"] = df_cleaned["Number of Employees"].apply(parse_employees)
        df_cleaned["Number of Employees"].fillna(df_cleaned["Number of Employees"].median(), inplace=True)
        print("✅ Imputed Number of Employees.")

    # Industry Groups
    if "Industry Groups" in df_cleaned.columns:
        df_cleaned = fill_by_grouped_mode(df_cleaned, "Industry Groups", "Industries")
        df_cleaned["Industry Groups"].fillna(df_cleaned["Industry Groups"].mode().iloc[0], inplace=True)
        print("✅ Imputed Industry Groups.")

    # Headquarters Regions
    if "Headquarters Regions" in df_cleaned.columns:
        def extract_region(location):
            if pd.isnull(location): return np.nan
            parts = location.split(",")
            if len(parts) == 3:
                return parts[1].strip()
            return np.nan

        df_cleaned['Extracted Region'] = df_cleaned['Headquarters Location'].apply(extract_region)
        df_cleaned['Headquarters Regions'].fillna(df_cleaned['Extracted Region'], inplace=True)
        df_cleaned['Headquarters Regions'].fillna('Unknown', inplace=True)
        df_cleaned.drop(columns=['Extracted Region'], inplace=True, errors='ignore')
        print("✅ Imputed Headquarters Regions.")

    # Step 3: Impute using Machine Learning models

    # Imputation function for regression
    def ml_impute_regressor(df, target_col, features, special_case=False):
        df_encoded = df.copy()
        df_encoded[target_col] = clean_numeric_col(df_encoded[target_col])

        # Clean features
        for col in features:
            if df_encoded[col].dtype == 'object':
                 df_encoded[col] = df_encoded[col].astype(str)
            else:
                df_encoded[col] = clean_numeric_col(df_encoded[col])

        cat_features = [col for col in features if df_encoded[col].dtype == 'object']

        train_df = df_encoded.dropna(subset=features + [target_col])
        predict_df = df_encoded[df_encoded[target_col].isnull()]

        if len(train_df) < 5 or predict_df.empty:
            print(f"ℹ️ Skipping ML imputation for {target_col} due to insufficient data or no missing values.")
            return df

        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

        # Preprocess and encode
        for col in cat_features:
            train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
            predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

        train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
        predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(train_df[features], train_df[target_col])
        y_pred = model.predict(predict_df[features])

        # If the target column should be an integer, round the predictions
        if 'Number' in target_col or target_col in ['IPqwery - Patents Granted', 'G2 Stack - Total Products Active']:
            y_pred = np.round(y_pred)

        df.loc[predict_df.index, target_col] = y_pred
        return df

    # Imputation function for classification
    def ml_impute_classifier(df, target_col, features):
        df_encoded = df.copy()

        cat_features = [col for col in features if df_encoded[col].dtype == 'object']

        train_df = df_encoded.dropna(subset=features + [target_col])
        predict_df = df_encoded[df_encoded[target_col].isnull()]

        if len(train_df) < 5 or predict_df.empty:
            print(f"ℹ️ Skipping ML imputation for {target_col} due to insufficient data or no missing values.")
            return df

        # Preprocess and encode
        for col in cat_features:
            train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
            predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
        predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(train_df[features], train_df[target_col])
        y_pred = model.predict(predict_df[features])

        df.loc[predict_df.index, target_col] = y_pred
        return df

    # List of columns to impute using ML
    impute_list_reg = {
        "Last Funding Amount": ["Estimated Revenue Range", "Number of Employees", "CB Rank (Company)", "Industries", "Funding Status"],
        "Total Funding Amount": ["Last Funding Amount", "Estimated Revenue Range", "Number of Employees", "CB Rank (Company)", "Industries", "Funding Status"],
        "Valuation at IPO": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Number of Articles": ["Number of Employees", "Total Funding Amount", "Estimated Revenue Range", "Industries", "Funding Status", "Number of Founders"],
        "Number of Investors": ["Estimated Revenue Range", "Industries", "Funding Status", "Total Funding Amount", "Number of Funding Rounds", "Last Funding Amount"],
        "BuiltWith - Active Tech Count": ["Number of Employees", "Total Funding Amount", "Estimated Revenue Range", "Industry Groups"],
        "G2 Stack - Total Products Active": ["Estimated Revenue Range", "Industries", "Funding Status", "Number of Employees", "Total Funding Amount"],
        "IPqwery - Patents Granted": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Number of Events": ["Number of Employees", "Number of Articles", "Number of Funding Rounds", "Estimated Revenue Range", "Industries"],
        "Number of Exits": ["Total Funding Amount", "Number of Employees", "Funding Status", "Industries", "Company Type"],
        "Price": ["Total Funding Amount", "Valuation at IPO", "Number of Employees", "Funding Status", "Industries"],
        "Number of Funding Rounds": ["Total Funding Amount", "Last Funding Amount", "Estimated Revenue Range", "Funding Status", "Industries"],
        "Number of Founders": ["Company Type", "Industries", "Number of Employees", "Total Funding Amount", "Number of Articles"]
    }

    impute_list_class = {
        "Estimated Revenue Range": ["Number of Employees", "CB Rank (Company)", "Funding Status", "Last Funding Amount", "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "Industries"],
        "Funding Status": ["Estimated Revenue Range", "Last Funding Amount", "CB Rank (Company)", "Number of Employees", "Industries"],
        "Last Funding Type": ["Funding Status", "Estimated Revenue Range", "Total Funding Amount", "Number of Funding Rounds", "Industries", "Last Funding Amount"],
        "Actively Hiring": ["Number of Employees", "Estimated Revenue Range", "Funding Status", "Company Type"]
    }

    # Order of Imputation:
    # 1. First, impute all columns that are used as features for others.
    # 2. Then, impute other columns.

    # Impute `Last Funding Amount` and `Total Funding Amount` first, as they are crucial for `Estimated Revenue Range`
    df_cleaned = ml_impute_regressor(df_cleaned, "Last Funding Amount", impute_list_reg["Last Funding Amount"])
    df_cleaned = ml_impute_regressor(df_cleaned, "Total Funding Amount", impute_list_reg["Total Funding Amount"])

    # New Logic: Overwrite Estimated Revenue Range based on Total Funding Amount where it was missing
    if "Estimated Revenue Range" in df_cleaned.columns and "Total Funding Amount" in df_cleaned.columns:
        null_revenue_mask = df_cleaned["Estimated Revenue Range"].isnull()

        # Apply new logic only to the previously missing rows
        df_cleaned.loc[null_revenue_mask & (df_cleaned["Total Funding Amount"] < 1000000), "Estimated Revenue Range"] = 'Less than $1M'
        df_cleaned.loc[null_revenue_mask & (df_cleaned["Total Funding Amount"].between(1000000, 10000000)), "Estimated Revenue Range"] = '$1M to $10M'
        df_cleaned.loc[null_revenue_mask & (df_cleaned["Total Funding Amount"] > 10000000), "Estimated Revenue Range"] = '$10M+'

        # Handle any remaining missing values with the old classifier model
        df_cleaned = ml_impute_classifier(df_cleaned, "Estimated Revenue Range", impute_list_class["Estimated Revenue Range"])
        print("✅ Imputed Estimated Revenue Range with new funding-based logic.")

    # Impute other core columns
    df_cleaned = ml_impute_classifier(df_cleaned, "Funding Status", impute_list_class["Funding Status"])
    df_cleaned = ml_impute_regressor(df_cleaned, "Number of Funding Rounds", impute_list_reg["Number of Funding Rounds"])
    df_cleaned = ml_impute_classifier(df_cleaned, "Last Funding Type", impute_list_class["Last Funding Type"])
    df_cleaned = ml_impute_regressor(df_cleaned, "Number of Founders", impute_list_reg["Number of Founders"])

    # Special case for SEMrush and Apptopia columns
    semrush_features = ["Estimated Revenue Range", "Industries", "Number of Employees", "Last Funding Amount"]
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Monthly Visits", semrush_features)
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Page Views / Visit", semrush_features + ["SEMrush - Monthly Visits"])
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Visit Duration", semrush_features + ["SEMrush - Page Views / Visit"])
    df_cleaned = ml_impute_regressor(df_cleaned, "SEMrush - Bounce Rate", semrush_features + ["SEMrush - Visit Duration"])
    print("✅ Imputed SEMrush columns.")

    apptopia_features = ["Estimated Revenue Range", "Industries", "Funding Status", "Number of Employees", "Apptopia - Number of Apps"]
    df_cleaned = ml_impute_regressor(df_cleaned, "Apptopia - Downloads Last 30 Days", apptopia_features)
    df_cleaned = ml_impute_regressor(df_cleaned, "Apptopia - Number of Apps", apptopia_features + ["Apptopia - Downloads Last 30 Days"])
    print("✅ Imputed Apptopia columns.")

    # Impute remaining columns
    remaining_cols = [
        "Valuation at IPO", "Number of Articles", "Number of Investors",
        "BuiltWith - Active Tech Count", "G2 Stack - Total Products Active",
        "IPqwery - Patents Granted", "Number of Events", "Number of Exits",
        "Price", "Actively Hiring"
    ]
    for col in remaining_cols:
        if col in impute_list_reg:
            df_cleaned = ml_impute_regressor(df_cleaned, col, impute_list_reg[col])
            print(f"✅ Imputed {col}.")
        elif col in impute_list_class:
            df_cleaned = ml_impute_classifier(df_cleaned, col, impute_list_class[col])
            print(f"✅ Imputed {col}.")

    # New Logic: Round numeric columns to nearest whole number
    numeric_cols_to_round = [
        "Number of Founders", "Number of Employees", "Number of Investors",
        "Number of Funding Rounds", "IPqwery - Patents Granted",
        "SEMrush - Monthly Visits", "SEMrush - Visit Duration", "SEMrush - Page Views / Visit",
        "SEMrush - Bounce Rate", "Number of Events", "BuiltWith - Active Tech Count",
        "G2 Stack - Total Products Active", "Number of Articles",
        "Number of Exits", "Apptopia - Downloads Last 30 Days", "Apptopia - Number of Apps"
    ]

    for col in numeric_cols_to_round:
        if col in df_cleaned.columns:
            # Check if the column is of a float type before rounding
            if pd.api.types.is_numeric_dtype(df_cleaned[col]):
                df_cleaned[col] = df_cleaned[col].round(0).astype('Int64')
    print("✅ Rounded specified numeric columns to the nearest whole number.")

    # Final check and save
    final_output_path = f"/content/{country_name}_final_filled.csv"
    df_cleaned.to_csv(final_output_path, index=False)
    print(f"✅ Final cleaned data for {country_name} saved to {final_output_path}")
    print(f"--- 🏁 {country_name} processing complete. ---\n")

# List of countries to process
countries_to_process = [
    ('Algeria', 'Algeria.csv'),
    ('Angola', 'Angola.csv'),
    ('Armenia', 'Armenia.csv'),
    ('Australia', 'Australia.csv'),
    ('Austria', 'Austria.csv'),
    ('Belarus', 'Belarus.csv'),
    ('Belize', 'Belize.csv'),
    ('British_Indian_Ocean_Territory', 'British_Indian_Ocean_Territory.csv'),
    ('Cameroon', 'Cameroon.csv'),
    ('Canada', 'Canada.csv'),
    ('Colombia', 'Colombia.csv'),
    ('Congo', 'Congo.csv'),
    ('Ecuador', 'Ecuador.csv'),
    ('Egypt', 'Egypt.csv'),
    ('Equatorial_Guinea', 'Equatorial_Guinea.csv'),
    ('Ethiopia', 'Ethiopia.csv'),
    ('Faroe_Islands', 'Faroe_Islands.csv'),
    ('Fiji', 'Fiji.csv'),
    ('Finland', 'Finland.csv'),
    ('Gabon', 'Gabon.csv'),
    ('Ghana', 'Ghana.csv'),
    ('Greece', 'Greece.csv'),
    ('Guinea', 'Guinea.csv'),
    ('Guyana', 'Guyana.csv'),
    ('Honduras', 'Honduras.csv'),
    ('Iceland', 'Iceland.csv'),
    ('Indonesia', 'Indonesia.csv'),
    ('Iraq', 'Iraq.csv'),
    ('Isle_of_Man', 'Isle_of_Man.csv'),
    ('Jersey', 'Jersey.csv'),
    ('Malawi', 'Malawi.csv'),
    ('Marshall_Islands', 'Marshall_Islands.csv'),
    ('Mauritania', 'Mauritania.csv'),
    ('Mauritius', 'Mauritius.csv')
]

# Run the process for each country
for country, file in countries_to_process:
    process_country_data(country, f"/content/{file}")

--- 🌎 Processing Italy data... ---
✅ Loaded data with shape: (8269, 31)
✅ Imputed CB Rank (Company).
✅ Imputed Founded Date.
✅ Imputed Company Type.
✅ Imputed Industries.
✅ Imputed Number of Employees.
✅ Imputed Industry Groups.
✅ Imputed Headquarters Regions.
ℹ️ Skipping ML imputation for Estimated Revenue Range due to insufficient data or no missing values.
✅ Imputed Estimated Revenue Range with new funding-based logic.
✅ Imputed SEMrush columns.
✅ Imputed Apptopia columns.
✅ Imputed Valuation at IPO.
✅ Imputed Number of Articles.
✅ Imputed Number of Investors.
✅ Imputed BuiltWith - Active Tech Count.
✅ Imputed G2 Stack - Total Products Active.
✅ Imputed IPqwery - Patents Granted.
✅ Imputed Number of Events.
✅ Imputed Number of Exits.
✅ Imputed Price.
✅ Imputed Actively Hiring.
✅ Rounded specified numeric columns to the nearest whole number.
✅ Final cleaned data for Italy saved to /content/Italy_final_filled.csv
--- 🏁 Italy processing complete. ---

--- 🌎 Processing Côte_dIvoire data

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import os
import warnings

warnings.filterwarnings('ignore')

def impute_actively_hiring(file_path):
    """
    Imputes the 'Actively Hiring' column using RandomForestClassifier
    and saves the updated data back to the same file.
    Includes a fallback logic for when there is insufficient training data.

    Args:
        file_path (str): The path to the CSV file to process.
    """
    print(f"--- Processing {os.path.basename(file_path)} ---")
    try:
        df_cleaned = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Error: {file_path} not found. Skipping.")
        return

    # --- Setup for Imputation ---
    target_col = "Actively Hiring"
    features = [
        "Number of Employees",
        "Estimated Revenue Range",
        "Funding Status",
        "Company Type"
    ]

    # Check if necessary columns exist
    if not all(col in df_cleaned.columns for col in features + [target_col]):
        print(f"⚠️ Required columns are missing in {os.path.basename(file_path)}. Skipping imputation.")
        return

    # Create a copy for encoding
    df_encoded = df_cleaned.copy()

    # --- Preprocessing and Encoding ---
    # Identify categorical features
    cat_features = [col for col in features if col in df_encoded.columns and df_encoded[col].dtype == 'object']

    # Prepare training data: remove rows where target is missing
    train_df = df_encoded.dropna(subset=features + [target_col]).copy()
    predict_df = df_encoded[df_encoded[target_col].isnull()].copy()

    if predict_df.empty:
        print("✅ No missing values for 'Actively Hiring'. Skipping imputation.")
        return

    if train_df.empty:
        print(f"⚠️ No sufficient data to train a model for '{target_col}'. Applying fallback logic.")
        # --- NEW FALLBACK LOGIC ---
        # Ensure Total Funding Amount is numeric for the fallback logic
        if "Total Funding Amount" in predict_df.columns:
            predict_df.loc[:, "Total Funding Amount"] = pd.to_numeric(
                predict_df["Total Funding Amount"].astype(str).str.replace('[,%$]', '', regex=True).str.strip(),
                errors='coerce'
            )
            # Apply the funding-based rule
            predict_df.loc[predict_df["Total Funding Amount"] < 10000, target_col] = 'No'
            predict_df.loc[predict_df["Total Funding Amount"] >= 10000, target_col] = 'Yes'
        else:
            print(f"⚠️ 'Total Funding Amount' is missing. Falling back to default 'No'.")
            predict_df.loc[:, target_col] = 'No'

        # Fill the original DataFrame with the predicted values and save
        df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = predict_df[target_col]
        df_cleaned.to_csv(file_path, index=False)
        print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")
        print(f"💾 Updated data saved back to {os.path.basename(file_path)}")
        print("-" * 30)
        return

    # Handle 'nan' values in categorical features for encoding
    for col in cat_features:
        train_df.loc[:, col] = train_df[col].astype(str).fillna('Unknown')
        predict_df.loc[:, col] = predict_df[col].astype(str).fillna('Unknown')

    # Use OrdinalEncoder to transform categorical features
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    train_df.loc[:, cat_features] = encoder.fit_transform(train_df[cat_features])
    predict_df.loc[:, cat_features] = encoder.transform(predict_df[cat_features])

    # --- Train and Predict ---
    X_train = train_df[features]
    y_train = train_df[target_col]
    X_pred = predict_df[features]

    # Train a RandomForestClassifier model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict the missing values
    y_pred = model.predict(X_pred)

    # --- Impute and Save ---
    # Fill missing values and save to the same file, overwriting the original
    df_cleaned.loc[df_cleaned[target_col].isnull(), target_col] = y_pred
    df_cleaned.to_csv(file_path, index=False)

    print(f"✅ Missing after imputation for {target_col}: {df_cleaned[target_col].isnull().sum()}")
    print(f"💾 Updated data saved back to {os.path.basename(file_path)}")
    print("-" * 30)


# List of countries to process, assuming cleaned files exist
countries_to_process = [
    'Algeria', 'Angola', 'Armenia', 'Australia', 'Austria', 'Belarus', 'Belize',
    'British_Indian_Ocean_Territory', 'Cameroon', 'Canada', 'Colombia', 'Congo',
    'Cayman_Islands', 'Cyprus', 'Côte_dIvoire', 'Germany', 'Italy', 'Ecuador',
    'Egypt', 'Equatorial_Guinea', 'Ethiopia', 'Faroe_Islands', 'Fiji', 'Finland',
    'Gabon', 'Ghana', 'Greece', 'Guinea', 'Guyana', 'Honduras', 'Iceland',
    'Indonesia', 'Iraq', 'Isle_of_Man', 'Jersey', 'Malawi', 'Marshall_Islands',
    'Mauritania', 'Mauritius'
]

# Run the process for each country
for country in countries_to_process:
    impute_actively_hiring(f'/content/{country}_final_filled.csv')


--- Processing Algeria_final_filled.csv ---
⚠️ No sufficient data to train a model for 'Actively Hiring'. Applying fallback logic.
✅ Missing after imputation for Actively Hiring: 0
💾 Updated data saved back to Algeria_final_filled.csv
------------------------------
--- Processing Angola_final_filled.csv ---
⚠️ No sufficient data to train a model for 'Actively Hiring'. Applying fallback logic.
✅ Missing after imputation for Actively Hiring: 4
💾 Updated data saved back to Angola_final_filled.csv
------------------------------
--- Processing Armenia_final_filled.csv ---
⚠️ No sufficient data to train a model for 'Actively Hiring'. Applying fallback logic.
✅ Missing after imputation for Actively Hiring: 0
💾 Updated data saved back to Armenia_final_filled.csv
------------------------------
--- Processing Australia_final_filled.csv ---
✅ No missing values for 'Actively Hiring'. Skipping imputation.
--- Processing Austria_final_filled.csv ---
✅ No missing values for 'Actively Hiring'. Skippin

In [14]:
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

def drop_specified_columns(file_path):
    """
    Drops a predefined list of columns from a CSV file and saves the updated
    data back to the same file, overwriting the original.

    Args:
        file_path (str): The path to the CSV file to process.
    """
    print(f"--- Processing {os.path.basename(file_path)} ---")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Error: {file_path} not found. Skipping.")
        return

    columns_to_drop = [
        "Company Type",
        "IPqwery - Patents Granted",
        "SEMrush - Monthly Visits",
        "SEMrush - Visit Duration",
        "SEMrush - Page Views / Visit",
        "SEMrush - Bounce Rate",
        "Number of Articles",
        "CB Rank (Company)",
        "Valuation at IPO",
        "Price",  # This column may not exist from the last script
        "Number of Exits",
        "Diversity Spotlight (US Headquarters Only)",
        "Industry Groups",
        "Apptopia - Downloads Last 30 Days",
        "Apptopia - Number of Apps",
        "SEMrush - Global Traffic Rank", # This column may not exist from the last script
        "Investor Type", # This column may not exist from the last script
        "Investment Stage", # This column may not exist from the last script
        "Number of Portfolio Organizations", # This column may not exist from the last script
        "IPqwery - Trademarks Registered", # This column may not exist from the last script
        "IPqwery - Most Popular Patent Class", # This column may not exist from the last script
        "IPqwery - Most Popular Trademark Class", # This column may not exist from the last script
        "G2 Stack - Total Products Active"
    ]

    # Identify which columns to drop that actually exist in the DataFrame
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

    if not existing_columns_to_drop:
        print(f"⚠️ None of the specified columns exist in {os.path.basename(file_path)}. Skipping column drop.")
        print("-" * 30)
        return

    # Drop the columns
    print(f"✅ Dropping columns: {existing_columns_to_drop}")
    df_updated = df.drop(columns=existing_columns_to_drop, errors='ignore')

    # Save the updated DataFrame back to the same file
    df_updated.to_csv(file_path, index=False)

    print(f"✅ Columns dropped. New shape: {df_updated.shape}")
    print(f"💾 Updated data saved back to {os.path.basename(file_path)}")
    print("-" * 30)


# List of countries to process, assuming cleaned files exist
countries_to_process = [
    'Algeria', 'Angola', 'Armenia', 'Australia', 'Austria', 'Belarus', 'Belize',
    'British_Indian_Ocean_Territory', 'Cameroon', 'Canada', 'Colombia', 'Congo',
    'Cayman_Islands', 'Cyprus', 'Côte_dIvoire', 'Germany', 'Italy', 'Ecuador',
    'Egypt', 'Equatorial_Guinea', 'Ethiopia', 'Faroe_Islands', 'Fiji', 'Finland',
    'Gabon', 'Ghana', 'Greece', 'Guinea', 'Guyana', 'Honduras', 'Iceland',
    'Indonesia', 'Iraq', 'Isle_of_Man', 'Jersey', 'Malawi', 'Marshall_Islands',
    'Mauritania', 'Mauritius'
]

# Run the process for each country
for country in countries_to_process:
    drop_specified_columns(f'/content/{country}_final_filled.csv')


--- Processing Algeria_final_filled.csv ---
✅ Dropping columns: ['Price']
✅ Columns dropped. New shape: (18, 16)
💾 Updated data saved back to Algeria_final_filled.csv
------------------------------
--- Processing Angola_final_filled.csv ---
✅ Dropping columns: ['Price']
✅ Columns dropped. New shape: (6, 16)
💾 Updated data saved back to Angola_final_filled.csv
------------------------------
--- Processing Armenia_final_filled.csv ---
✅ Dropping columns: ['Price']
✅ Columns dropped. New shape: (49, 16)
💾 Updated data saved back to Armenia_final_filled.csv
------------------------------
--- Processing Australia_final_filled.csv ---
✅ Dropping columns: ['Price']
✅ Columns dropped. New shape: (3004, 16)
💾 Updated data saved back to Australia_final_filled.csv
------------------------------
--- Processing Austria_final_filled.csv ---
✅ Dropping columns: ['Price']
✅ Columns dropped. New shape: (509, 16)
💾 Updated data saved back to Austria_final_filled.csv
------------------------------
--- Pr